In [10]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
import time

In [11]:
def get_percentile(x, t='latest',val=0):
    if t=='latest':
        x=x.dropna().values
        val=x[-1]
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    else:
        return  np.percentile(np.sort(x),val)

In [19]:
#holdings=['RHHBY','BIG','DOCU','CLX']
holdings=['CDNS']

In [24]:
holdings='CMG,SPY,DOCU,ZS,FB,OLLI,SHOP,MBUU,ADBE,BL,QQQ,MDB,BLK,CDNS,MELI,AZN,NOW,NYT,ECL,CWH,BYND,DDOG,BIG,TREX,TW,DKNG'



In [25]:
holdings=holdings.split(',')

In [26]:
#ticker, #1wkret #1wkmean, 4wk, 4wkmean,k, kchange, kchangemean,k_change percentile, std, stdmean
res=[[],[],[],[],[],[],[],[],[],[],[]]

In [27]:
c=1
for ticker in holdings:
    #print(ticker)
    c+=1
    if c==5:
        c=1
        time.sleep(60)
    d={'function': 'TIME_SERIES_WEEKLY_ADJUSTED','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'outputsize':'full'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )
    spy=[[],[],[],[],[],[],[]]
    for key, val in response.json()['Weekly Adjusted Time Series'].items():
        spy[0].append(float(val['1. open']))
        spy[1].append(float(val['2. high']))
        spy[2].append(float(val['3. low']))
        spy[3].append(float(val['4. close']))
        spy[4].append(float(val['5. adjusted close']))
        #spy[5].append(float(val['6. volume']))
        #spy[6].append(float(val['7. dividend amount']))
    t=list(response.json()['Weekly Adjusted Time Series'].keys())
    time.sleep(1)
    '''
    try:
        if (pd.to_datetime(t[0]).date()==pd.Timestamp.today().date())==False:
            #print('Notlive')
            d={'token':"pk_9469e57bc642462d905e4dc0003523ab",'chartIEXOnly':'False','chartLast':'1'}
            response = requests.get(
            'https://cloud.iexapis.com/stable/stock/{}/intraday-prices?'.format(ticker)#token={}&chartIEXOnly=True&chartLast=1'.format(holdings[0],'pk_9469e57bc642462d905e4dc0003523ab'),
            ,params=d,
            )
            
            spy[0].insert(0,float(response.json()[0]['open']))
            spy[1].insert(0,float(response.json()[0]['high']))
            spy[2].insert(0,float(response.json()[0]['low']))
            spy[3].insert(0,float(response.json()[0]['close']))
            spy[4].insert(0,float(response.json()[0]['close']))
            t.insert(0,str(response.json()[0]['date']))

            #spy[0].append(float(t.insert(0,response2.json()[0]['date'])))
    except:
        print(ticker,'hi')
    '''    
        
    spy=pd.DataFrame({'Date':t,
                 'open':spy[0],
                  'high':spy[1],
                  'low':spy[2],
                  'close':spy[3],
                  'adj_close':spy[4]#,
                #  'volume':spy[5],
                #  'dividend':spy[6],
                 })
    spy=spy.astype({'Date': 'datetime64'})
    spy=spy.sort_values(by='Date')   
    spy['ret4wk']=spy.adj_close/spy.adj_close.shift(3)-1
    spy['highest_high']=spy.high.rolling(12).max()
    spy['lowest_low']=spy.low.rolling(12).min()
    spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
    spy['k']=spy.k.rolling(3).mean()
    spy['k_change'] = spy.k/spy.k.shift(3)-1
    spy['rstd']=spy.adj_close.rolling(12).std()
    spy['ret1wk']=spy.adj_close/spy.adj_close.shift(1)-1
    d={'ticker':ticker,'k_change':spy.k_change.values[-1]}
    res[0].append(ticker)
    res[1].append(spy.ret1wk.values[-1])
    res[2].append(spy.ret1wk.rolling(3).mean().values[-1])
    res[3].append(spy.ret4wk.values[-1])
    res[4].append(spy.ret4wk.rolling(3).mean().values[-1])
    res[5].append(spy.k.values[-1])
    res[6].append(spy.k_change.values[-1])
    res[7].append(spy.k_change.rolling(3).mean().values[-1])
    res[8].append(get_percentile(spy.k_change))
    res[9].append(spy.rstd.values[-1])
    res[10].append(spy.rstd.rolling(3).mean().values[-1])
    
    
    
    

In [28]:
res=pd.DataFrame({'ticker':res[0], 
            'ret1wk':res[1],
             'ret1wk_mean':res[2],
             'ret4wk':res[3],
             'ret4wk_mean':res[4],
             'k':res[5],
             'k_change':res[6],     
             'k_change_mean':res[7],
             'k_change_percentile':res[8],
             'rstd':res[9],
             'rstd_mean':res[10]
             }
            )

In [29]:
res.sort_values(by='ret1wk_mean',ascending=False)

,ticker,ret1wk,ret1wk_mean,ret4wk,ret4wk_mean,k,k_change,k_change_mean,k_change_percentile,rstd,rstd_mean
19,CWH,0.118059,0.170440,0.596429,1.016963,0.948669,0.944629,1.323272,85.06,6.069306,4.850982
25,DKNG,0.147029,0.129802,0.423881,0.460431,0.953364,0.166034,0.188427,71.43,7.629194,6.395038
21,DDOG,-0.027930,0.119301,0.372511,0.534219,0.925557,0.184024,0.461474,50.00,15.040111,12.537622
22,BIG,0.108541,0.112472,0.375813,0.372564,0.983346,0.272392,0.579745,68.58,7.862671,6.554432
5,OLLI,0.037963,0.086177,0.276357,0.251884,0.986623,0.071189,0.019247,58.40,16.531688,13.995618
11,MDB,-0.017342,0.072355,0.226472,0.288819,0.965408,0.218797,0.306042,72.50,36.612517,30.806194
7,MBUU,0.003617,0.063082,0.197208,0.339461,0.872135,0.854260,1.073110,88.54,9.122762,8.264665
23,TREX,0.002271,0.037353,0.112173,0.280629,0.958093,0.329211,0.579686,72.47,21.516239,18.689528
2,DOCU,-0.030451,0.035356,0.106297,0.196377,0.948209,0.036394,0.046224,55.91,20.507265,18.532319
0,CMG,-0.036658,0.033110,0.098474,0.135423,0.962607,0.054722,0.146211,58.88,169.428385,153.902053


In [30]:
res.sort_values(by='ret4wk',ascending=False)

,ticker,ret1wk,ret1wk_mean,ret4wk,ret4wk_mean,k,k_change,k_change_mean,k_change_percentile,rstd,rstd_mean
19,CWH,0.118059,0.170440,0.596429,1.016963,0.948669,0.944629,1.323272,85.06,6.069306,4.850982
25,DKNG,0.147029,0.129802,0.423881,0.460431,0.953364,0.166034,0.188427,71.43,7.629194,6.395038
22,BIG,0.108541,0.112472,0.375813,0.372564,0.983346,0.272392,0.579745,68.58,7.862671,6.554432
21,DDOG,-0.027930,0.119301,0.372511,0.534219,0.925557,0.184024,0.461474,50.00,15.040111,12.537622
5,OLLI,0.037963,0.086177,0.276357,0.251884,0.986623,0.071189,0.019247,58.40,16.531688,13.995618
11,MDB,-0.017342,0.072355,0.226472,0.288819,0.965408,0.218797,0.306042,72.50,36.612517,30.806194
7,MBUU,0.003617,0.063082,0.197208,0.339461,0.872135,0.854260,1.073110,88.54,9.122762,8.264665
23,TREX,0.002271,0.037353,0.112173,0.280629,0.958093,0.329211,0.579686,72.47,21.516239,18.689528
2,DOCU,-0.030451,0.035356,0.106297,0.196377,0.948209,0.036394,0.046224,55.91,20.507265,18.532319
0,CMG,-0.036658,0.033110,0.098474,0.135423,0.962607,0.054722,0.146211,58.88,169.428385,153.902053


In [89]:
res.sort_values(by='ret1wk_mean',ascending=False)

,ticker,ret1wk,ret1wk_mean,ret4wk,ret4wk_mean,k_change,k_change_mean,rstd,rstd_mean
2,MBUU,-0.021702,0.035501,0.105023,0.212375,0.367513,0.418247,4.642872,4.469681
1,AZN,0.029935,0.012033,0.036294,0.014843,-0.062579,-0.245810,0.781145,0.714343
0,SPY,-0.005153,0.000521,0.001356,0.021040,0.268086,0.749622,5.008670,4.658034
3,MSFT,-0.008349,-0.000480,-0.001592,0.010671,0.110782,0.205791,2.064047,2.186029
4,NFLX,-0.032412,-0.014429,-0.042910,-0.012232,-0.365573,-0.104620,7.978609,8.842592


In [9]:
msft=spy.copy(deep=True)

In [14]:
msft['ret4wk']=msft.adj_close/msft.adj_close.shift(3)-1
spy['ret4wk']=spy.adj_close/spy.adj_close.shift(3)-1

In [16]:
spy['highest_high']=spy.high.rolling(12).max()
spy['lowest_low']=spy.low.rolling(12).min()
spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
spy['k']=spy.k.rolling(3).mean()
spy['k_change'] = spy.k/spy.k.shift(3)-1

msft['highest_high']=msft.high.rolling(12).max()
msft['lowest_low']=msft.low.rolling(12).min()
msft['k']=(msft.close-msft.lowest_low)/(msft.highest_high-msft.lowest_low)
msft['k']=msft.k.rolling(3).mean()
msft['k_change'] = msft.k/msft.k.shift(3)-1




In [22]:
spy['rstd']=spy.adj_close.rolling(12).std()
msft['rstd']=msft.adj_close.rolling(12).std()

In [25]:
msft['ret1wk']=msft.adj_close/msft.adj_close.shift(1)-1
spy['ret1wk']=spy.adj_close/spy.adj_close.shift(1)-1

In [26]:
msft.tail(10)

,Date,open,high,low,close,adj_close,volume,dividend,ret4wk,highest_high,lowest_low,k,k_change,rstd,ret1wk
9,2020-03-20,140.00,150.15,135.0000,137.35,137.35,421347734.0,0.0,-0.152213,190.70,135.00,0.226001,-0.641828,12.881111,-0.135239
8,2020-03-27,137.01,156.66,132.5200,149.70,149.70,358866845.0,0.0,-0.073467,190.70,132.52,0.242002,-0.424578,13.538858,0.089916
7,2020-04-03,152.44,164.78,150.0100,153.83,153.83,290191457.0,0.0,-0.031480,190.70,132.52,0.234586,-0.239293,13.876028,0.027589
6,2020-04-09,160.32,170.00,157.5800,165.14,165.14,229630744.0,0.0,0.202330,190.70,132.52,0.407414,0.802706,13.859286,0.073523
5,2020-04-17,164.35,180.00,162.3000,178.60,178.60,238965670.0,0.0,0.193053,190.70,132.52,0.572992,1.367711,14.477445,0.081507
4,2020-04-24,176.63,178.75,166.1102,174.55,174.55,194621072.0,0.0,0.134694,190.70,132.52,0.691704,1.948616,14.676007,-0.022676
3,2020-05-01,176.59,180.40,169.3900,174.57,174.57,212119968.0,0.0,0.057103,190.70,132.52,0.745732,0.830404,13.910374,0.000115
2,2020-05-08,174.49,185.00,173.8000,184.68,184.68,158579959.0,0.0,0.034043,188.18,132.52,0.794096,0.385877,13.888795,0.057914
1,2020-05-15,183.15,187.51,175.6800,183.16,183.16,196126640.0,0.0,0.049327,187.51,132.52,0.860257,0.243678,14.351698,-0.008230
0,2020-05-19,185.75,186.60,183.4900,183.63,183.63,61664938.0,0.0,0.051899,187.51,132.52,0.929152,0.245960,15.227043,0.002566


In [38]:
spy.ret4wk.rolling(3).mean().values[-1]
spy.k_change.rolling(3).mean().values[-1]
spy.rstd.rolling(3).mean().values[-1]

22.84021899844733

In [47]:
spy.ret1wk.values[-1]

0.019875646220483656